In [4]:
# Ingestion


def leggi_file_txt(nome_file):

    lista_righe = []
    try:
        with open(nome_file, 'r') as file:
            for riga in file:
                lista_righe.append(riga[:-1].split(' '))


    except FileNotFoundError:
        print(f"Errore: Il file {nome_file} non è stato trovato.")
    except ValueError as e:
        print(f"Errore durante la conversione dei dati: {e}")

    for riga in lista_righe:
        disposizione=[]
        for carattere in riga[0]:
            print(1)
    return  lista_righe


nome_file = 'input_day10.txt'
nome_file_test= 'input_day10_test.txt'
data= leggi_file_txt(nome_file)
data_test=leggi_file_txt(nome_file_test)

In [11]:
#Part 1

out_1=0

input=data_test




['(', '1', ',', '3', ')']